# ReverberatorEstimator notebook
Jupyter Notebook for the Master Thesis work Parametric Tuning of Extended Reverberation Algorithm Using Neural Networks by Søren V.K. Lyster

## Import needed packages

In [ ]:
import matplotlib.pyplot as plt
import IPython
import tensorflow as tf
print(tf.__version__)
import tensorflow.keras as tfk
from ReverberatorEstimator import loss, models, utils, config
import warnings
warnings.filterwarnings('ignore')
import time
import os
import datetime
import IPython

## Setup environment

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Setup variables for the notebook

In [ ]:
k = config.k
sample_rate = k['sample_rate']
sample_length = k['sample_length']
num_epochs = k['epochs']
num_processors = k['n_processors']
steps_per_epoch = k['steps_per_epoch']
batch_size = steps_per_epoch * num_processors
epsilon = k['epsilon']
learning_rate = k['learning_rate']
dry_audio_path = k['dry_audio_path']
wet_audio_path = k['wet_audio_path']
vst_path = k['vst_path']
time_loss_weight = k['time_loss_weight']
spectral_loss_weight = k['spectral_loss_weight']
envelope_loss_weight = k['envelope_loss_weight']
echo_density_loss_weight = k['echo_density_loss_weight']
use_multiscale = k['use_multiscale']
num_params = k['n_parameters']
parameter_map = k['parameter_map']
non_trainable_parameters = k['non_trainable_parameters']
pretrained_weights = k['pretrained_weights']
checkpoint_path = k['checkpoint_path']

print(parameter_map)

## Setup dataset for batch training

In [ ]:
x_train, y_train = utils.get_dataset(dry_audio_path, wet_audio_path, batch_size, resample=True, old_sample_rate=48000, new_sample_rate=sample_rate)

## Create layers, create partial models, and compile full model 

In [ ]:
model, parameter_model, processor = models.get_models(sample_length, sample_rate, num_params, num_processors, 
                                                vst_path, epsilon, parameter_map, non_trainable_parameters, 
                                                pretrained_weights)

reverberation_loss = loss.reverberationLoss(sample_rate=sample_rate,
    spectral_loss_weight=spectral_loss_weight,
    spectral_loss_type='L1',
    time_loss_weight=time_loss_weight,
    time_loss_type='L1',
    envelope_loss_weight=envelope_loss_weight,
    envelope_loss_type='L1',
    echo_density_weight=echo_density_loss_weight,
    echo_density_type='L1',
    use_multiscale=use_multiscale,
    )                     

optimizer = tfk.optimizers.Adam(learning_rate=learning_rate) 
model.compile(optimizer=optimizer, loss=reverberation_loss, run_eagerly=True)

## Print model summaries

In [ ]:
parameter_model.summary()
model.summary()

## Setup checkpoint and callbacks

In [ ]:
checkpoint_dir = os.path.dirname(checkpoint_path)

model_cp = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True,
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                              factor=0.5,
                              patience=500,
                              cooldown=1,
                              verbose=1,
                              mode='auto',
                              min_lr=1e-10)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tfk.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

## Restore from previous checkpoint if it exists

In [ ]:
try:
    model.load_weights(checkpoint_path)
except:
    print("No previous checkpoints found at %s" % checkpoint_path)

## Run model and save data before training for analysis and debugging

In [ ]:
input_audio = tf.reshape(x_train[0], (1, sample_length))
target_audio = tf.reshape(y_train[0], (1, sample_length))

audio_pre = (model.call(input_audio)).numpy()[0]
old_params = parameter_model(input_audio).numpy()[0]
print(old_params)

## Run the model.fit to begin training. 

In [ ]:
start_time = time.time()
history = model.fit(x_train, y_train, verbose=1, epochs=num_epochs, steps_per_epoch=steps_per_epoch,
         callbacks=[model_cp, lr_callback])
print("Training took %d seconds" % (time.time() - start_time))

## Plot training loss metrics

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7,5))
ax.plot(history.history['loss'])
ax.set_title('loss')
ax.set_ylabel('loss')
ax.set_xlabel('epoch')
ax.legend(['train', 'test'], loc='upper left')

## Run a forward pass and get the output audio of the trained model

In [ ]:
model.load_weights(checkpoint_path)
output_audio = model(input_audio)
processor.print_current_parameters()

## Display the output audio from before training the model
This is done to inspect the changes the training has done

In [ ]:
utils.plot_single(audio_pre, sample_rate, sample_length)

## Plot the output audio against the target audio

In [ ]:
utils.plot_output_and_target(output_audio, target_audio, sample_rate)
IPython.display.display(IPython.display.Audio(output_audio, rate=sample_rate, autoplay=True))
IPython.display.display(IPython.display.Audio(target_audio, rate=sample_rate))

## Plot the loss function differences

In [ ]:
utils.plot_differences(output_audio, target_audio, sample_rate, weights=[time_loss_weight,spectral_loss_weight,envelope_loss_weight,echo_density_loss_weight])

In [ ]:
utils.plot_differences(output_audio, tf.reshape(tf.convert_to_tensor(audio_pre), (1,sample_length)), sample_rate)

## Print the parameters
These parameters are from the parameter model subpart of the full model. These values are transferable to the FDN reverberator plugin at [https://github.com/VoggLyster/Reverberator]

In [ ]:
params = parameter_model(input_audio).numpy()[0]
print('New parameter set: ', params)
plt.stem(params)
plt.ylim(0,1)

## Plot the parameter differences of before and after training
This shows the movement of the parameters after training and can give a good picture of the momentum of the training loop

In [ ]:
param_diff = params - old_params
print('Parameter set difference: ', param_diff)
plt.stem(param_diff)
plt.ylim(-1,1)

## Generate MUSHRA-ready audio files

In [ ]:
audio_data, audio_names = utils.generate_MUSHRA_ready_audio(vst_path, params, sample_rate)
for i in range(len(audio_data)):
    print(audio_names[i])
    IPython.display.display(IPython.display.Audio(audio_data[i], rate=sample_rate))
utils.write_audio_files(audio_data, audio_names, 'MUSHRA_audio/Wet/AbletonReverb', sample_rate)